In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import dask
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cmcrameri import cm
import babet as bb
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
import metpy.calc as mpcalc 
from metpy.units import units

sns.set_theme(style="white")
sns.set_style("white")

random.seed(10)
# inidates = ['2023-10-11', '2023-10-15', '2023-10-17']
inidates = ['2023-10-11', '2023-10-13', '2023-10-15', '2023-10-17'] # TODO: change as needed
experiments = ['pi', 'curr', 'incr'] # TODO: change as needed

dask.config.set(**{'array.slicing.split_large_chunks': True})

# Import data

I import the data so that the different scenarios are entries in a dictionary and the initialisation dates are included in the xarray along with the other data dimensions. Overall, I will then have a xarray with dimensions
- time
- initialisation date (called 'inidate' in data)
- longitude
- latitude
- pressure level (called 'level' in data)
- ensemble members (called 'number' in data)

This script also imports the surface level data which is structured in the same way, except for the absence of the pressure level dimension.
Variables I need here are specific humidity, and horizontal winds u and v. I mostly plot the advection on 850hPa so only that pressure level should be necessary here.

In [2]:
# Import forecast data 
base_dir = '/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/MED-R/EXP/{}/EU025/pl/pf' # TODO: change as needed
exp = {}
for experiment in experiments:
    exp[experiment] = xr.open_mfdataset(os.path.join(base_dir.format(experiment), '*.nc'), preprocess=bb.Data.preproc_ds)

In [3]:
# Import forecast data at surface
base_dir = '/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/MED-R/EXP/{}/EU025/sfc/pf' # TODO: change as needed
exp_sfc = {}
for experiment in experiments:
    exp_sfc[experiment] = xr.open_mfdataset(os.path.join(base_dir.format(experiment), '*.nc'), preprocess=bb.Data.preproc_ds)

I also import ERA5 data from a file which contains the data for October 2023. The data is 3-hourly and also loaded into an xarray. 

In [4]:
# Import ERA5 data
era5_dir = '/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/ERA5/EU025/pl/' # TODO: change as needed
era5 = xr.open_mfdataset(os.path.join(era5_dir, '*.nc'))
era5 = era5.sel(time=slice('2023-10-17 00', '2023-10-23 00'))

In [5]:
era5

<xarray.Dataset>
Dimensions:    (longitude: 289, latitude: 163, level: 23, time: 49)
Coordinates:
  * longitude  (longitude) float32 -27.0 -26.75 -26.5 -26.25 ... 44.5 44.75 45.0
  * latitude   (latitude) float32 73.5 73.25 73.0 72.75 ... 33.5 33.25 33.0
  * level      (level) int32 200 225 250 300 350 400 ... 900 925 950 975 1000
  * time       (time) datetime64[ns] 2023-10-17 ... 2023-10-23
Data variables:
    z          (time, level, latitude, longitude) float32 dask.array<chunksize=(49, 23, 163, 289), meta=np.ndarray>
    t          (time, level, latitude, longitude) float32 dask.array<chunksize=(49, 23, 163, 289), meta=np.ndarray>
    q          (time, level, latitude, longitude) float32 dask.array<chunksize=(49, 23, 163, 289), meta=np.ndarray>
    pv         (time, level, latitude, longitude) float32 dask.array<chunksize=(49, 23, 163, 289), meta=np.ndarray>
    v          (time, level, latitude, longitude) float32 dask.array<chunksize=(49, 23, 163, 289), meta=np.ndarray>
    u          (time, level, latitude, longitude) float32 dask.array<chunksize=(49, 23, 163, 289), meta=np.ndarray>
    vo         (time, level, latitude, longitude) float32 dask.array<chunksize=(49, 23, 163, 289), meta=np.ndarray>
    w          (time, level, latitude, longitude) float32 dask.array<chunksize=(49, 23, 163, 289), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-10 11:00:59 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

# Calculate advection

In [6]:
def calc_advection_q(ds, level=850):
    '''
    Calculate scalar horizontal advection of q
    on a pressure level

    Parameters
    ----------
    ds : xarray.Dataset
        Dataset containing u, v and q variables
    level : int, optional
        Pressure level to calculate advection, by default 850

    Returns
    -------
    xarray.DataArray
        Scalar advection
    '''
    # Calculate gradient of q
    dqdlon =ds.sel(level=level).q.diff(n=2, dim='longitude') / bb.met.Met.haversine(ds.sel(level=level).latitude, ds.sel(level=level).longitude.values[:-2], ds.sel(level=level).latitude, ds.sel(level=level).longitude[2:])
    dqdlat = ds.sel(level=level).q.diff(n=2, dim='latitude') / bb.met.Met.haversine(ds.sel(level=level).latitude.values[:-2], ds.sel(level=level).longitude, ds.sel(level=level).latitude[2:], ds.sel(level=level).longitude)
    
    # Calculate advection
    adv = (ds.sel(level=level).u.isel(latitude=slice(1, -1), longitude=slice(1, -1)) * dqdlon) + (ds.sel(level=level).v.isel(latitude=slice(1, -1), longitude=slice(1, -1)) * dqdlat)
    return adv

In [7]:
# Calculate scalar advection
level = 850
adv = {}
for experiment in experiments:
    adv[experiment] = calc_advection_q(exp[experiment], level=level) 

# Calculate unit vector of wind
u = {}
v = {}
for experiment in experiments:
    u[experiment] = (exp[experiment].u / np.sqrt(exp[experiment].u**2 + exp[experiment].v**2)).sel(level=level).isel(latitude=slice(1, -1), longitude=slice(1, -1))
    v[experiment] = (exp[experiment].v / np.sqrt(exp[experiment].u**2 + exp[experiment].v**2)).sel(level=level).isel(latitude=slice(1, -1), longitude=slice(1, -1))

In [8]:
# Advection in ERA5
adv_era5 = calc_advection_q(era5, level=level)

# Calculate unit vector of wind in ERA5
u_era5 = (era5.u / np.sqrt(era5.u**2 + era5.v**2)).sel(level=level).isel(latitude=slice(1, -1), longitude=slice(1, -1))
v_era5 = (era5.v / np.sqrt(era5.u**2 + era5.v**2)).sel(level=level).isel(latitude=slice(1, -1), longitude=slice(1, -1))

In [9]:
# Plot settings

# UK
lat_max = 62
lat_min = 47
lon_min = -12
lon_max = 5

# #Europe
# lat_max = 70
# lat_min = 33
# lon_min = -27
# lon_max = 25

euroatlantic = [lon_min-13, lon_max, lat_min-5, lat_max+6]
uk = [-11, 10, 48, 70]
northsea = [-17, 20, 40, 70]

# Time series for current climate

In [ ]:
time = '2023-10-20 09'
level = 850

latitude = exp['curr'].latitude.values[1:-1]
longitude = exp['curr'].longitude.values[1:-1]

times = ['2023-10-19 12', '2023-10-20 12', '2023-10-21 12', '2023-10-22 00']

fig, ax = plt.subplots(4, 4, figsize=(15, 15), subplot_kw={'projection': ccrs.PlateCarree()})

for ini, inidate in enumerate(inidates):
    for i, time in enumerate(times):
        ax[ini][i].set_extent(uk, crs=ccrs.PlateCarree())
        ax[ini][i].add_feature(cfeature.COASTLINE.with_scale('50m'), edgecolor='white', linewidth=0.5)

        # Plot scalar advection
        (adv['curr'].sel(time=time, inidate=inidate).mean('number')).plot(vmin=-1e-4, vmax=1e-4, 
                                                                        cmap=cm.devon 
                                                                        ax=ax[ini][i],
                                                                        transform=ccrs.PlateCarree(),
                                                                        cbar_kwargs={"label": "Advection (g/kg/s)"})

        # Plot wind vectors
        stride = 4
        llon, llat = np.meshgrid(longitude[::stride], latitude[::stride])
        ax[ini][i].quiver(llon, llat, 
                u['curr'].sel(time=time, inidate=inidate).mean('number')[::stride, ::stride], 
                v['curr'].sel(time=time, inidate=inidate).mean('number')[::stride, ::stride], 
                transform=ccrs.PlateCarree(), scale=20, color='white')
        ax[ini][i].set_title(f'{time}')
plt.suptitle(f'Advection and wind direction, {level} hPa')

plt.savefig('../figures/A02_advection_curr_timeseries.png', dpi=600)
plt.savefig('../figures/A02_advection_curr_timeseries.pdf') 

# Comparison between experiments

In [ ]:
# figure and map setup
experiments = ['pi', 'curr', 'incr']
fs = 18
projection = ccrs.PlateCarree()

adv_min = 0
adv_max = 0.0000005

fig = plt.figure(1, figsize=(20, 11))
lead_times = ['inidate 2023-10-15', 'inidate 2023-10-17'] # TODO: change as needed
inidates = ['2023-10-15', '2023-10-17'] # TODO: change as needed
starttime = '2023-10-19 00'
endtime = '2023-10-22 00'

# EPS data ------------------    

for i, inidate in enumerate(inidates):
    for e, experiment in enumerate(experiments):
        latitude = adv['curr'].latitude.values
        longitude = adv['curr'].longitude.values

        if experiment in ['pi', 'incr']:  # plot difference for counterfactual scenarios
            adv_vals = ((adv[experiment].sel(time=slice(starttime, endtime), inidate=inidate).mean(['number', 'time'])) - (adv['curr'].sel(time=slice(starttime, endtime), inidate=inidate).mean(['number', 'time']))).squeeze().values
        else: 
            adv_vals = (adv['curr'].sel(time=slice(starttime, endtime), inidate=inidate).mean(['number', 'time'])).values

        ax = plt.subplot(3,3, i+1+e*3,projection = projection)
        ax.set_extent(uk, projection)
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'), color = 'white', zorder = 14)

        # advection as shading
        if experiment =='curr':
            clevs_adv = np.linspace(adv_min, adv_max, 11)  # 17
            cf = ax.contourf(longitude, latitude, adv_vals, clevs_adv, cmap=cm.devon,
                            transform=projection, zorder = 10, extend = 'both')
        else: 
            clevs_adv = np.linspace(adv_max*(-0.1), adv_max*0.1, 10)
            cf_diff = ax.contourf(longitude, latitude, adv_vals, clevs_adv, cmap=cm.bam,
                                  transform=projection, zorder = 10, extend = 'both')
        
        # Plot wind vectors
        latitude = u[experiment].latitude.values
        longitude = u[experiment].longitude.values
        stride = 4
        llon, llat = np.meshgrid(longitude[::stride], latitude[::stride])
        ax.quiver(llon, llat, 
                u[experiment].sel(time=slice(starttime, endtime), inidate=inidate).mean(['number', 'time'])[::stride, ::stride], 
                v[experiment].sel(time=slice(starttime, endtime), inidate=inidate).mean(['number', 'time'])[::stride, ::stride], 
                transform=ccrs.PlateCarree(), scale=20, color='white')

        # rectangle for Aberdeenshire box
        rectangle = patches.Rectangle((-4, 55.5), 2, 2, linewidth=2, 
                                      edgecolor='k', 
                                      facecolor='none',
                                      transform=projection)
        ax.add_patch(rectangle)
        rectangle.set_zorder(17)

# ERA5 or analysis data ----------------------
latitude = adv_era5.latitude.values
longitude = adv_era5.longitude.values

adv_vals = (adv_era5.sel(time=slice(starttime, endtime)).mean('time')*1e6).values
ax = plt.subplot(3,3,6,projection = projection)
ax.set_extent(uk, projection)
ax.add_feature(cfeature.COASTLINE.with_scale('50m'), color = 'white', zorder = 14)
ax.add_feature(cfeature.OCEAN, zorder=12, color = 'white')  # zorder > 10

# advection as shading
clevs_adv = np.linspace(adv_min, adv_max, 11)  # 17
cf = ax.contourf(longitude, latitude, adv_vals, clevs_adv, cmap=cm.devon,
                transform=projection, zorder = 10, extend = 'both')

# Plot wind vectors
stride = 4
latitude = u_era5.latitude.values
longitude = u_era5.longitude.values
llon, llat = np.meshgrid(longitude[::stride], latitude[::stride])
ax.quiver(llon, llat, 
        u_era5.sel(time=slice(starttime, endtime)).mean('time')[::stride, ::stride], 
        v_era5.sel(time=slice(starttime, endtime)).mean('time')[::stride, ::stride], 
        transform=ccrs.PlateCarree(), scale=20, color='white')

# rectangle for Aberdeenshire box
rectangle = patches.Rectangle((-4, 55.5), 2, 2, linewidth=2, 
                                edgecolor='k', 
                                facecolor='none',
                                transform=projection)
ax.add_patch(rectangle)
rectangle.set_zorder(17)

# Other figure settings -----------------
ax = plt.subplot(3,3,3)
ax.axis('off')  # removes subplot frame
cax = ax.inset_axes([0.2, 0.02, 0.1, 0.95])  # creates inset, [x0,y0, width, height]
cbar = fig.colorbar(cf, cax=cax, label='advection (g/kg/s)', extend = 'max', shrink=0.8)
cbar.set_label(label='advection (g/kg/s)', size=fs)
cbar.ax.tick_params(labelsize=fs-3)

cax_diff = ax.inset_axes([0.6, 0.02, 0.1, 0.95])  # creates inset, [x0,y0, width, height]
cbar_diff = fig.colorbar(cf_diff, cax=cax_diff, label='advection difference (g/kg/s)', extend = 'both', shrink=0.8)
cbar_diff.set_label(label='advection difference (g/kg/s)', size=fs)
cbar_diff.ax.tick_params(labelsize=fs-3)
plt.figtext(-0.02, 0.82, 'pi', rotation='vertical', size=fs)
plt.figtext(-0.02, 0.48, 'curr', rotation='vertical', size=fs)
plt.figtext(-0.02, 0.15, 'fut', rotation='vertical', size=fs)
plt.suptitle(f'Advection at {level}', size = fs)
plt.tight_layout()

plt.savefig('../figures/A02_advection.png', dpi=600)
plt.savefig('../figures/A02_advection.pdf')   